In [0]:
%run ./02_autoloader_ingestion

In [0]:
%run ./04_custom_ingestion

In [0]:
# Databricks notebook source
# MAGIC %run "./02_autoloader_ingestion"
'''
log("🚀 Starting Retail Org Auto Loader ingestion pipeline...")

# Step 1: Identify valid and unsupported folders/files
supported_sources, unsupported_sources = list_source_items(SOURCE_BASE)
log(f"Found {len(supported_sources)} supported and {len(unsupported_sources)} unsupported sources.")

# Step 2: Ingest supported datasets
for src in supported_sources:
    folder_name = src.rstrip("/").split("/")[-1]
    tgt = f"{VOLUME_BASE}{folder_name}_autoloader/"
    log(f"Ingesting {folder_name} -> {tgt}")
    ingest_with_autoloader(src, tgt)

# Step 3: Copy unsupported (non-data) files for archival
for src in unsupported_sources:
    folder_name = src.rstrip("/").split("/")[-1]
    tgt = f"{VOLUME_BASE}{folder_name}_raw_copy/"
    log(f"Copying unsupported files from {src} -> {tgt}")
    try:
        dbutils.fs.cp(src, tgt, recurse=True)
        log(f"✅ Copied {src} successfully.")
    except Exception as e:
        log(f"❌ Failed to copy {src}: {e}")

log("🎉 All ingestion and copies completed.")'''


In [0]:
display(dbutils.fs.ls("dbfs:/databricks-datasets/retail-org/sales_orders/"))

In [0]:
# Databricks notebook source
# MAGIC %run "./02_autoloader_ingestion"
# MAGIC %run "./04_custom_ingestion"  # 👈 Added this line to include custom ingestion logic

# COMMAND ----------

log("🚀 Starting Retail Org ingestion pipeline (Auto Loader + Custom PySpark)...")

# Step 1️⃣: Identify valid and unsupported folders/files
supported_sources, unsupported_sources = list_source_items(SOURCE_BASE)
log(f"Found {len(supported_sources)} supported and {len(unsupported_sources)} unsupported sources.")

# Step 2️⃣: AUTOLOADER INGESTION
for src in supported_sources:
    folder_name = src.rstrip("/").split("/")[-1]
    tgt = f"{VOLUME_BASE}{folder_name}_autoloader/"
    log(f"⚡ Auto Loader Ingesting: {folder_name} -> {tgt}")
    try:
        ingest_with_autoloader(src, tgt)
    except Exception as e:
        log(f"❌ Auto Loader failed for {src}: {e}")

# Step 3️⃣: CUSTOM INGESTION (manual PySpark logic)
# You can specify a few folders you want to test custom ingestion for
custom_folders = [
    "customers", 
    "products",
    "suppliers"
]

for folder_name in custom_folders:
    src = f"{SOURCE_BASE}{folder_name}/"
    tgt = f"{VOLUME_BASE}{folder_name}_custom_ingest/"
    log(f"🔥 Custom PySpark ingestion for {folder_name} -> {tgt}")
    try:
        custom_ingest(src, tgt, load_type="full")  # You can change to incremental if needed
    except Exception as e:
        log(f"❌ Custom ingestion failed for {src}: {e}")

# Step 4️⃣: Copy unsupported (non-data) files for archival
for src in unsupported_sources:
    folder_name = src.rstrip("/").split("/")[-1]
    tgt = f"{VOLUME_BASE}{folder_name}_raw_copy/"
    log(f"📂 Copying unsupported files from {src} -> {tgt}")
    try:
        dbutils.fs.cp(src, tgt, recurse=True)
        log(f"✅ Copied {src} successfully.")
    except Exception as e:
        log(f"❌ Failed to copy {src}: {e}")

log("🎉 All Auto Loader and Custom Ingestion tasks completed successfully.")
